## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os
import sys

sys.path.append('/Users/insiktuser/Desktop/UCB - Analysis Projects/World_Weather_Analysis')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
new_city_data_df = pd.read_csv('/Users/insiktuser/Desktop/UCB - Analysis Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv')
new_city_data_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sao Filipe,CV,14.90,-24.50,80.47,73,100,9.80,overcast clouds
1,1,Salta,AR,-24.79,-65.41,73.40,69,40,3.36,scattered clouds
2,2,Vao,NC,-22.67,167.48,73.38,82,63,18.54,broken clouds
3,3,Clyde River,CA,70.47,-68.59,17.60,78,75,16.11,broken clouds
4,4,Bengkulu,ID,-3.80,102.27,84.15,67,99,5.82,overcast clouds
5,5,Inta,RU,66.03,60.17,22.86,92,100,11.07,overcast clouds
6,6,Avarua,CK,-21.21,-159.78,75.20,73,56,12.75,broken clouds
7,7,Rikitea,PF,-23.12,-134.97,74.93,83,98,19.71,light rain
8,8,Korla,CN,41.76,86.15,56.10,36,0,12.64,clear sky
9,9,Georgetown,MY,5.41,100.34,84.20,79,40,6.93,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = new_city_data_df.loc[(new_city_data_df["Max Temp"] <= max_temp) & \
                                       (new_city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sao Filipe,CV,14.90,-24.50,80.47,73,100,9.80,overcast clouds
4,4,Bengkulu,ID,-3.80,102.27,84.15,67,99,5.82,overcast clouds
6,6,Avarua,CK,-21.21,-159.78,75.20,73,56,12.75,broken clouds
9,9,Georgetown,MY,5.41,100.34,84.20,79,40,6.93,scattered clouds
13,13,Busselton,AU,-33.65,115.33,86.00,21,79,5.73,broken clouds
14,14,Sola,VU,-13.88,167.55,80.58,80,27,15.93,scattered clouds
15,15,Airai,TL,-8.93,125.41,80.35,55,62,3.24,light rain
19,19,Atuona,PF,-9.80,-139.03,78.84,74,21,20.51,few clouds
20,20,Tarauaca,BR,-8.16,-70.77,77.81,76,86,3.36,overcast clouds
33,33,Garowe,SO,8.41,48.48,79.65,41,36,5.93,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        202
City           202
Country        202
Lat            202
Lng            202
Max Temp       202
Humidity       202
Cloudiness     202
Wind Speed     202
Description    202
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,80.47,overcast clouds,14.90,-24.50,
4,Bengkulu,ID,84.15,overcast clouds,-3.80,102.27,
6,Avarua,CK,75.20,broken clouds,-21.21,-159.78,
9,Georgetown,MY,84.20,scattered clouds,5.41,100.34,
13,Busselton,AU,86.00,broken clouds,-33.65,115.33,
14,Sola,VU,80.58,scattered clouds,-13.88,167.55,
15,Airai,TL,80.35,light rain,-8.93,125.41,
19,Atuona,PF,78.84,few clouds,-9.80,-139.03,
20,Tarauaca,BR,77.81,overcast clouds,-8.16,-70.77,
33,Garowe,SO,79.65,scattered clouds,8.41,48.48,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
     
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           202
Country        202
Max Temp       202
Description    202
Lat            202
Lng            202
Hotel Name     202
dtype: int64

In [27]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,80.47,overcast clouds,14.90,-24.50,Tortuga B&B
4,Bengkulu,ID,84.15,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
6,Avarua,CK,75.20,broken clouds,-21.21,-159.78,Paradise Inn
9,Georgetown,MY,84.20,scattered clouds,5.41,100.34,Cititel Penang
13,Busselton,AU,86.00,broken clouds,-33.65,115.33,Observatory Guest House
14,Sola,VU,80.58,scattered clouds,-13.88,167.55,Ulkel Guest House
15,Airai,TL,80.35,light rain,-8.93,125.41,Juvinal Martin
19,Atuona,PF,78.84,few clouds,-9.80,-139.03,Villa Enata
20,Tarauaca,BR,77.81,overcast clouds,-8.16,-70.77,Flat Temporada
33,Garowe,SO,79.65,scattered clouds,8.41,48.48,Curubo Hotel


In [28]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template =  """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>and<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined